<a href="https://colab.research.google.com/github/dan-a-iancu/airm/blob/master/Project_Portfolio_Management/Project_Portfolio_Management_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implements the solution to the **Project Portfolio Management** mini-case. It assumes you are familiar with the case and the model.

____
# Basic Setup

Import useful modules, read the data and store it in data frames, and set up some useful Python lists. You may want to expand this section and make sure you understand how the data is organized, and also read the last part where the Python lists are created, as these may be very useful when you build your model.

In [ ]:
#@markdown We first import some useful modules. 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# import numpy
import numpy as np
import urllib.request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

from IPython.display import display

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker 

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

# some modules to create local directories for CBC (to avoid issues with solving multiple models)
import os
def new_local_directory(name):
    full_path = os.path.join(".", name)
    os.makedirs(full_path, exist_ok=True)
    return full_path

# install the latest version of seaborn for nicer graphics
#!pip install --prefix {sys.prefix} seaborn==0.11.0  &> /dev/null
import seaborn as sns

# Ignore useless some warnings
import warnings
warnings.simplefilter(action="ignore")

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
Completed successfully!


## Load the case data into Pandas data frames

We first download an Excel file with all the data from Github.

In [ ]:
#@markdown Download the entire data as an Excel file from Github

url_Excel = 'https://github.com/dan-a-iancu/airm/blob/master/Project_Portfolio_Management/Project_Data.xlsx?raw=true'
local_file = "Portfolio_Project_Data.xlsx"   # name of local file where you want to store the downloaded file
urllib.request.urlretrieve(url_Excel, local_file)    # download from website and save it locally

('Portfolio_Project_Data.xlsx', <http.client.HTTPMessage at 0x7f8dc1b27510>)

Read in and store the data in suitable dataframes.

In [ ]:
#@markdown Create a dataframe with the information on projects

# Read in the information about the available projects
projectData = pd.read_excel("Portfolio_Project_Data.xlsx", sheet_name = "Projects", index_col = 0)

# Have a look:
display(projectData)


,Capital_Required,Location,Impact,Risks,Engineers,Field_Workers,Support
Project,,,,,,,
A,4015,South America,10,10,15,80,15
B,3623,South America,7,10,5,35,15
C,3847,South America,4,7,10,30,10
D,4058,Asia,4,4,5,35,4
E,11182,Asia,10,7,10,50,5
F,4858,Asia,4,4,5,45,5
G,5379,Asia,4,4,5,40,5
H,16203,Africa,10,7,5,100,15
I,1839,Africa,1,4,2,70,10


In [ ]:
#@markdown Create a dataframe with the information on available resources (this is useful in **Q5**)

# Read in the information about the available projects
resourceData = pd.read_excel("Portfolio_Project_Data.xlsx", sheet_name = "Resources", index_col = 0)

# Have a look:
display(resourceData)

,January-March,April-June,July-September,October-December
Engineers,20,20,12,20
Field_Workers,120,150,150,120
Support,50,40,40,50


Also set up any other problem data/parameters, such as the initial capital available.

In [ ]:
initialCapital = 25000

## Create Python lists based on the data-frames

__NOTE__: Make sure you understand what the __lists__ created here are! These will be very helpful when creating the model.

In [ ]:
#@markdown Some useful lists for building all the models

# the list with project names (A,B, ...)
allProjects = list(projectData.index)
print("This is the list of all the project names:")
print(allProjects)

# the unique locations / continents
allLocations = list(projectData["Location"].unique())
print("\nThis is the list of unique locations:")
print(allLocations)

This is the list of all the project names:
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

This is the list of unique locations:
['South America', 'Asia', 'Africa']


In [ ]:
#@markdown The following lists will be useful in **Q5**

# the list with periods when the projects could be scheduled
allPeriods = list(resourceData.columns)
print("These are periods when the projects could be scheduled:")
print(allPeriods)

# the types of resources needed
allResources = list(resourceData.index)
print("\nThese are the unique resources needed to execute the projects:")
print(allResources)

These are periods when the projects could be scheduled:
['January-March', 'April-June', 'July-September', 'October-December']

These are the unique resources needed to execute the projects:
['Engineers', 'Field_Workers', 'Support']


**Q1**